Databricks code Source : https://docs.databricks.com/en/connect/external-systems/elasticsearch.html


# Elasticsearch

<img src="https://static-www.elastic.co/v3/assets/bltefdd0b53724fa2ce/blt5ebe80fb665aef6b/5ea8c8f26b62d4563b6ecec2/brand-elasticsearch-220x130.svg" width="300">

1. Launch a cluster in your workspace or choose an existing cluster.
2. Once the new cluster is running, go to the "Libraries" tab of that cluster, and click "Install new" -> choose "Maven" -> enter the maven coordinates `org.elasticsearch:elasticsearch-spark-30_2.12:8.4.3` -> click "Install". If running into errors like `org.elasticsearch.hadoop.EsHadoopIllegalArgumentException: Cannot detect ES version` while the ES connection is verified, consider install newer versions that matches your ES service.
3. Once the installation has finished, attach this notebook to the cluster, and run write and/or read operations against your Elasticsearch cluster

**Important**: In the following cells, replace `<ip-address>`, `<port>`, `<ssl>`, `<hostname>` and `<index>` with your Elasticsearch configuration

##Install Elasticseach on Databricks worker

In [0]:
%sh 

curl -fsSL https://artifacts.elastic.co/GPG-KEY-elasticsearch | sudo gpg --dearmor -o /usr/share/keyrings/elasticsearch-archive-keyring.gpg


gpg: cannot open '/dev/tty': No such device or address
(23) Failed writing body


In [0]:
%sh 
echo "deb [signed-by=/usr/share/keyrings/elasticsearch-archive-keyring.gpg] https://artifacts.elastic.co/packages/8.x/apt stable main" | sudo tee /etc/apt/sources.list.d/elastic-8.x.list


deb [signed-by=/usr/share/keyrings/elasticsearch-archive-keyring.gpg] https://artifacts.elastic.co/packages/8.x/apt stable main


In [0]:
%sh 
sudo apt-get update


Get:1 https://artifacts.elastic.co/packages/8.x/apt stable InRelease [10.4 kB]
Hit:2 https://repos.azul.com/zulu/deb stable InRelease
Hit:3 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:6 https://artifacts.elastic.co/packages/8.x/apt stable/main amd64 Packages [126 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]
Hit:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [4,435 kB]
Get:10 http://archive.ubuntu.com/ubuntu focal-updates/multiverse amd64 Packages [33.5 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,558 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [4,124 kB]
Fetched 10.4 MB in 3s (3,158 kB/s)
Reading package lists...


In [0]:
%sh 

sudo apt-get install elasticsearch

#The generated password for the elastic built-in superuser is : D5Gro*bR5us2hYt06dXI


Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  elasticsearch
0 upgraded, 1 newly installed, 0 to remove and 51 not upgraded.
Need to get 606 MB of archives.
After this operation, 1,168 MB of additional disk space will be used.
Get:1 https://artifacts.elastic.co/packages/8.x/apt stable/main amd64 elasticsearch amd64 8.15.1 [606 MB]
debconf: delaying package configuration, since apt-utils is not installed
Fetched 606 MB in 20s (29.8 MB/s)
Selecting previously unselected package elasticsearch.
(Reading database ... 
(Reading database ... 5%
(Reading database ... 10%
(Reading database ... 15%
(Reading database ... 20%
(Reading database ... 25%
(Reading database ... 30%
(Reading database ... 35%
(Reading database ... 40%
(Reading database ... 45%
(Reading database ... 50%
(Reading database ... 55%
(Reading database ... 60%
(Reading database ... 65%
(Reading database ... 70%
(Reading database ... 75%
(Reading 

## Start Elasticserach service 

In [0]:
%sh 

sudo systemctl enable elasticsearch

Created symlink /etc/systemd/system/multi-user.target.wants/elasticsearch.service → /lib/systemd/system/elasticsearch.service.


In [0]:
%sh 

sudo systemctl start elasticsearch

In [0]:
%sh 

sudo systemctl status elasticsearch

● elasticsearch.service - Elasticsearch
     Loaded: loaded (/lib/systemd/system/elasticsearch.service; enabled; vendor preset: enabled)
     Active: active (running) since Mon 2024-09-16 07:41:12 UTC; 38min ago
       Docs: https://www.elastic.co
   Main PID: 2893 (java)
     CGroup: /system.slice/elasticsearch.service
             ├─2893 /usr/share/elasticsearch/jdk/bin/java -Xms4m -Xmx64m -XX:+UseSerialGC -Dcli.name=server -Dcli.script=/usr/share/elasticsearch/bin/elasticsearch -Dcli.libs=lib/tools/server-cli -Des.path.home=/usr/share/elasticsearch -Des.path.conf=/etc/elasticsearch -Des.distribution.type=deb -cp /usr/share/elasticsearch/lib/*:/usr/share/elasticsearch/lib/cli-launcher/* org.elasticsearch.launcher.CliToolLauncher -p /var/run/elasticsearch/elasticsearch.pid --quiet
             ├─2964 /usr/share/elasticsearch/jdk/bin/java -Des.networkaddress.cache.ttl=60 -Des.networkaddress.cache.negative.ttl=10 -Djava.security.manager=allow -XX:+AlwaysPreTouch -Xss1m -Djava.awt.headle

In [0]:
%sh 

lsb_release -d


Description:	Ubuntu 20.04.6 LTS


In [0]:
%sh 

sudo tail -f /var/log/elasticsearch/elasticsearch.log

[2024-09-16T07:59:42,023][WARN ][o.e.h.n.Netty4HttpServerTransport] [0916-072530-anrzknxi-10-172-214-67] received plaintext http traffic on an https channel, closing connection Netty4HttpChannel{localAddress=/127.0.0.1:9200, remoteAddress=/127.0.0.1:37548}
[2024-09-16T07:59:52,488][WARN ][o.e.h.n.Netty4HttpServerTransport] [0916-072530-anrzknxi-10-172-214-67] received plaintext http traffic on an https channel, closing connection Netty4HttpChannel{localAddress=/127.0.0.1:9200, remoteAddress=/127.0.0.1:48968}
[2024-09-16T07:59:55,906][WARN ][o.e.h.n.Netty4HttpServerTransport] [0916-072530-anrzknxi-10-172-214-67] received plaintext http traffic on an https channel, closing connection Netty4HttpChannel{localAddress=/127.0.0.1:9200, remoteAddress=/127.0.0.1:48972}
[2024-09-16T07:59:58,867][WARN ][o.e.h.n.Netty4HttpServerTransport] [0916-072530-anrzknxi-10-172-214-67] received plaintext http traffic on an https channel, closing connection Netty4HttpChannel{localAddress=/127.0.0.1:9200, remo

## Backup elasticsearch.yml

In [0]:
%sh 

mkdir /tmp

mkdir: cannot create directory ‘/tmp’: File exists


In [0]:
%sh tail /etc/elasticsearch/elasticsearch.yml


# Allow HTTP API connections from anywhere
# Connections are encrypted and require user authentication
http.host: 0.0.0.0

# Allow other nodes to join the cluster from anywhere
# Connections are encrypted and mutually authenticated
#transport.host: 0.0.0.0

#----------------------- END SECURITY AUTO CONFIGURATION -------------------------


In [0]:
%sh 
cp /etc/elasticsearch/elasticsearch.yml /tmp/elasticsearch_cp.yml

In [0]:
%sh

echo '# ======================== Elasticsearch Configuration =========================
#
# NOTE: Elasticsearch comes with reasonable defaults for most settings.
#       Before you set out to tweak and tune the configuration, make sure you
#       understand what are you trying to accomplish and the consequences.
#
# The primary way of configuring a node is via this file. This template lists
# the most important settings you may want to configure for a production cluster.
#
# Please consult the documentation for further information on configuration options:
# https://www.elastic.co/guide/en/elasticsearch/reference/index.html
#
# ---------------------------------- Cluster -----------------------------------
#
# Use a descriptive name for your cluster:
#
#cluster.name: my-application
#
# ------------------------------------ Node ------------------------------------
#
# Use a descriptive name for the node:
#
#node.name: node-1
#
# Add custom attributes to the node:
#
#node.attr.rack: r1
#
# ----------------------------------- Paths ------------------------------------
#
# Path to directory where to store the data (separate multiple locations by comma):
#
path.data: /var/lib/elasticsearch
#
# Path to log files:
#
path.logs: /var/log/elasticsearch
#
# ----------------------------------- Memory -----------------------------------
#
# Lock the memory on startup:
#
#bootstrap.memory_lock: true
#
# Make sure that the heap size is set to about half the memory available
# on the system and that the owner of the process is allowed to use this
# limit.
#
# Elasticsearch performs poorly when the system is swapping the memory.
#
# ---------------------------------- Network -----------------------------------
#
# By default Elasticsearch is only accessible on localhost. Set a different
# address here to expose this node on the network:
#
#network.host: 192.168.0.1
#
# By default Elasticsearch listens for HTTP traffic on the first free port it
# finds starting at 9200. Set a specific HTTP port here:
#
#http.port: 9200
#
# For more information, consult the network module documentation.
#
# --------------------------------- Discovery ----------------------------------
#
# Pass an initial list of hosts to perform discovery when this node is started:
# The default list of hosts is ["127.0.0.1", "[::1]"]
#
#discovery.seed_hosts: ["host1", "host2"]
#
# Bootstrap the cluster using an initial set of master-eligible nodes:
#
#cluster.initial_master_nodes: ["node-1", "node-2"]
#
# For more information, consult the discovery and cluster formation module documentation.
#
# ---------------------------------- Various -----------------------------------
#
# Allow wildcard deletion of indices:
#
#action.destructive_requires_name: false

#----------------------- BEGIN SECURITY AUTO CONFIGURATION -----------------------
#
# The following settings, TLS certificates, and keys have been automatically      
# generated to configure Elasticsearch security features on 16-09-2024 07:39:27
#
# --------------------------------------------------------------------------------

# Enable security features
xpack.security.enabled: false

xpack.security.enrollment.enabled: true

# Enable encryption for HTTP API client connections, such as Kibana, Logstash, and Agents
xpack.security.http.ssl:
  enabled: true
  keystore.path: certs/http.p12

# Enable encryption and mutual authentication between cluster nodes
xpack.security.transport.ssl:
  enabled: true
  verification_mode: certificate
  keystore.path: certs/transport.p12
  truststore.path: certs/transport.p12
# Create a new cluster with the current node only
# Additional nodes can still join the cluster later
cluster.initial_master_nodes: ["0916-072530-anrzknxi-10-172-214-67"]

# Allow HTTP API connections from anywhere
# Connections are encrypted and require user authentication
http.host: 0.0.0.0

# Allow other nodes to join the cluster from anywhere
# Connections are encrypted and mutually authenticated
#transport.host: 0.0.0.0

#----------------------- END SECURITY AUTO CONFIGURATION -------------------------
' > /tmp/elasticsearch_default.yml




In [0]:
%sh 
ls /tmp/e*

/tmp/elasticsearch_cp.yml
/tmp/elasticsearch_default.yml

/tmp/elasticsearch-1011582052528151951:
geoip-databases
jansi-2.4.0-eaf0436a468662be-libjansi.so
jansi-2.4.0-eaf0436a468662be-libjansi.so.lck


In [0]:
cat /tmp/elasticsearch_default.yml

# ======================== Elasticsearch Configuration =========================
#
# NOTE: Elasticsearch comes with reasonable defaults for most settings.
#       Before you set out to tweak and tune the configuration, make sure you
#       understand what are you trying to accomplish and the consequences.
#
# The primary way of configuring a node is via this file. This template lists
# the most important settings you may want to configure for a production cluster.
#
# Please consult the documentation for further information on configuration options:
# https://www.elastic.co/guide/en/elasticsearch/reference/index.html
#
# ---------------------------------- Cluster -----------------------------------
#
# Use a descriptive name for your cluster:
#
#cluster.name: my-application
#
# ------------------------------------ Node ------------------------------------
#
# Use a descriptive name for the node:
#
#node.name: node-1
#
# Add custom attributes to the node:
#
#node.attr.rack: r1
#
# ----

## Create new elasticsearch.yml without SSL

In [0]:
%sh

#xpack.security.enabled set to false

echo '
# ======================== Elasticsearch Configuration =========================
#
# NOTE: Elasticsearch comes with reasonable defaults for most settings.
#       Before you set out to tweak and tune the configuration, make sure you
#       understand what are you trying to accomplish and the consequences.
#
# The primary way of configuring a node is via this file. This template lists
# the most important settings you may want to configure for a production cluster.
#
# Please consult the documentation for further information on configuration options:
# https://www.elastic.co/guide/en/elasticsearch/reference/index.html
#
# ---------------------------------- Cluster -----------------------------------
#
# Use a descriptive name for your cluster:
#
#cluster.name: my-application
#
# ------------------------------------ Node ------------------------------------
#
# Use a descriptive name for the node:
#
#node.name: node-1
#
# Add custom attributes to the node:
#
#node.attr.rack: r1
#
# ----------------------------------- Paths ------------------------------------
#
# Path to directory where to store the data (separate multiple locations by comma):
#
path.data: /var/lib/elasticsearch
#
# Path to log files:
#
path.logs: /var/log/elasticsearch
#
# ----------------------------------- Memory -----------------------------------
#
# Lock the memory on startup:
#
#bootstrap.memory_lock: true
#
# Make sure that the heap size is set to about half the memory available
# on the system and that the owner of the process is allowed to use this
# limit.
#
# Elasticsearch performs poorly when the system is swapping the memory.
#
# ---------------------------------- Network -----------------------------------
#
# By default Elasticsearch is only accessible on localhost. Set a different
# address here to expose this node on the network:
#
#network.host: 192.168.0.1
#
# By default Elasticsearch listens for HTTP traffic on the first free port it
# finds starting at 9200. Set a specific HTTP port here:
#
http.port: 9200
#
# For more information, consult the network module documentation.
#
# --------------------------------- Discovery ----------------------------------
#
# Pass an initial list of hosts to perform discovery when this node is started:
# The default list of hosts is ["127.0.0.1", "[::1]"]
#
#discovery.seed_hosts: ["host1", "host2"]
#
# Bootstrap the cluster using an initial set of master-eligible nodes:
#
#cluster.initial_master_nodes: ["node-1", "node-2"]
#
# For more information, consult the discovery and cluster formation module documentation.
#
# ---------------------------------- Various -----------------------------------
#
# Allow wildcard deletion of indices:
#
#action.destructive_requires_name: false

#----------------------- BEGIN SECURITY AUTO CONFIGURATION -----------------------
#
# The following settings, TLS certificates, and keys have been automatically      
# generated to configure Elasticsearch security features on 16-09-2024 07:39:27
#
# --------------------------------------------------------------------------------

# Enable security features
xpack.security.enabled: false

xpack.security.enrollment.enabled: true

# Enable encryption for HTTP API client connections, such as Kibana, Logstash, and Agents
xpack.security.http.ssl:
  enabled: true
  keystore.path: certs/http.p12

# Enable encryption and mutual authentication between cluster nodes
xpack.security.transport.ssl:
  enabled: true
  verification_mode: certificate
  keystore.path: certs/transport.p12
  truststore.path: certs/transport.p12
# Create a new cluster with the current node only
# Additional nodes can still join the cluster later
cluster.initial_master_nodes: ["0916-072530-anrzknxi-10-172-214-67"]

# Allow HTTP API connections from anywhere
# Connections are encrypted and require user authentication
http.host: 0.0.0.0

# Allow other nodes to join the cluster from anywhere
# Connections are encrypted and mutually authenticated
#transport.host: 0.0.0.0

#----------------------- END SECURITY AUTO CONFIGURATION -------------------------
' > /tmp/elasticsearch_new.yml



In [0]:
%sh 

cat /tmp/elasticsearch_new.yml


# ======================== Elasticsearch Configuration =========================
#
# NOTE: Elasticsearch comes with reasonable defaults for most settings.
#       Before you set out to tweak and tune the configuration, make sure you
#       understand what are you trying to accomplish and the consequences.
#
# The primary way of configuring a node is via this file. This template lists
# the most important settings you may want to configure for a production cluster.
#
# Please consult the documentation for further information on configuration options:
# https://www.elastic.co/guide/en/elasticsearch/reference/index.html
#
# ---------------------------------- Cluster -----------------------------------
#
# Use a descriptive name for your cluster:
#
#cluster.name: my-application
#
# ------------------------------------ Node ------------------------------------
#
# Use a descriptive name for the node:
#
#node.name: node-1
#
# Add custom attributes to the node:
#
#node.attr.rack: r1
#
# ---


## Replace elasticsearch.yml and restart

In [0]:
%sh 
sudo rm /etc/elasticsearch/elasticsearch.yml 

In [0]:
%sh
sudo cp /tmp/elasticsearch_new.yml /etc/elasticsearch/elasticsearch.yml 

In [0]:
%sh 

cat /etc/elasticsearch/elasticsearch.yml 


# ======================== Elasticsearch Configuration =========================
#
# NOTE: Elasticsearch comes with reasonable defaults for most settings.
#       Before you set out to tweak and tune the configuration, make sure you
#       understand what are you trying to accomplish and the consequences.
#
# The primary way of configuring a node is via this file. This template lists
# the most important settings you may want to configure for a production cluster.
#
# Please consult the documentation for further information on configuration options:
# https://www.elastic.co/guide/en/elasticsearch/reference/index.html
#
# ---------------------------------- Cluster -----------------------------------
#
# Use a descriptive name for your cluster:
#
#cluster.name: my-application
#
# ------------------------------------ Node ------------------------------------
#
# Use a descriptive name for the node:
#
#node.name: node-1
#
# Add custom attributes to the node:
#
#node.attr.rack: r1
#
# ---

In [0]:
%sh 

sudo systemctl restart elasticsearch

In [0]:
%sh

sudo systemctl status elasticsearch

● elasticsearch.service - Elasticsearch
     Loaded: loaded (/lib/systemd/system/elasticsearch.service; enabled; vendor preset: enabled)
     Active: active (running) since Mon 2024-09-16 09:01:04 UTC; 1min 18s ago
       Docs: https://www.elastic.co
   Main PID: 11953 (java)
     CGroup: /system.slice/elasticsearch.service
             ├─11953 /usr/share/elasticsearch/jdk/bin/java -Xms4m -Xmx64m -XX:+UseSerialGC -Dcli.name=server -Dcli.script=/usr/share/elasticsearch/bin/elasticsearch -Dcli.libs=lib/tools/server-cli -Des.path.home=/usr/share/elasticsearch -Des.path.conf=/etc/elasticsearch -Des.distribution.type=deb -cp /usr/share/elasticsearch/lib/*:/usr/share/elasticsearch/lib/cli-launcher/* org.elasticsearch.launcher.CliToolLauncher -p /var/run/elasticsearch/elasticsearch.pid --quiet
             ├─12043 /usr/share/elasticsearch/jdk/bin/java -Des.networkaddress.cache.ttl=60 -Des.networkaddress.cache.negative.ttl=10 -Djava.security.manager=allow -XX:+AlwaysPreTouch -Xss1m -Djava.awt.

In [0]:
%sh 

journalctl -xe

-- Logs begin at Mon 2024-09-16 07:26:22 UTC, end at Mon 2024-09-16 09:01:05 UTC. --
Sep 16 07:36:32 0916-072530-anrzknxi-10-172-214-67 ntpd[84]: 216.31.17.12 local addr 10.172.214.67 -> <null>
Sep 16 07:36:33 0916-072530-anrzknxi-10-172-214-67 ntpd[84]: adj_systime: Operation not permitted
Sep 16 07:36:34 0916-072530-anrzknxi-10-172-214-67 ntpd[84]: adj_systime: Operation not permitted
Sep 16 07:36:35 0916-072530-anrzknxi-10-172-214-67 ntpd[84]: adj_systime: Operation not permitted
Sep 16 07:36:36 0916-072530-anrzknxi-10-172-214-67 ntpd[84]: adj_systime: Operation not permitted
Sep 16 07:36:36 0916-072530-anrzknxi-10-172-214-67 ntpd[84]: 173.255.232.93 local addr 10.172.214.67 -> <null>
Sep 16 07:36:37 0916-072530-anrzknxi-10-172-214-67 ntpd[84]: adj_systime: Operation not permitted
Sep 16 07:36:37 0916-072530-anrzknxi-10-172-214-67 ntpd[84]: 207.246.65.226 local addr 10.172.214.67 -> <null>
Sep 16 07:36:38 0916-072530-anrzknxi-10-172-214-67 ntpd[84]: adj_systime: Operation not permit

In [0]:
%sh 
netstat | grep 9200

## Test elasticsearch with curl

In [0]:
%sh 
curl -X GET "localhost:9200"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   559  100   559    0     0  15971      0 --:--:-- --:--:-- --:--:-- 15971
{
  "name" : "0916-072530-anrzknxi-10-172-214-67",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "ZuQf3GKpQFiS_NrD0llKLw",
  "version" : {
    "number" : "8.15.1",
    "build_flavor" : "default",
    "build_type" : "deb",
    "build_hash" : "253e8544a65ad44581194068936f2a5d57c2c051",
    "build_date" : "2024-09-02T22:04:47.310170297Z",
    "build_snapshot" : false,
    "lucene_version" : "9.11.1",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [0]:
%sh 

curl -X GET "localhost:9200/_cluster/health"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   389  100   389    0     0   2080      0 --:--:-- --:--:-- --:--:--  2080
100   389  100   389    0     0   2080      0 --:--:-- --:--:-- --:--:--  2069
{"cluster_name":"elasticsearch","status":"green","timed_out":false,"number_of_nodes":1,"number_of_data_nodes":1,"active_primary_shards":0,"active_shards":0,"relocating_shards":0,"initializing_shards":0,"unassigned_shards":0,"delayed_unassigned_shards":0,"number_of_pending_tasks":0,"number_of_in_flight_fetch":0,"task_max_waiting_in_queue_millis":0,"active_shards_percent_as_number":100.0}

In [0]:
%sh

curl -X PUT "localhost:9200/curl_index?pretty"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    86  100    86    0     0     98      0 --:--:-- --:--:-- --:--:--    98
100    86  100    86    0     0     98      0 --:--:-- --:--:-- --:--:--    98
{
  "acknowledged" : true,
  "shards_acknowledged" : true,
  "index" : "curl_index"
}


In [0]:
%sh

curl -X POST "localhost:9200/curl_index/_bulk?pretty" -H 'Content-Type: application/json' -d'
{ "index": { "_id": "1" } }
{ "name": "Bilbo", "age": 50 }
{ "index": { "_id": "2" } }
{ "name": "Gandalf", "age": 1000 }
{ "index": { "_id": "3" } }
{ "name": "Thorin", "age": 195 }
'


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  1228    0  1044  100   184   2367    417 --:--:-- --:--:-- --:--:--  2784
{
  "errors" : false,
  "took" : 400,
  "items" : [
    {
      "index" : {
        "_index" : "curl_index",
        "_id" : "1",
        "_version" : 1,
        "result" : "created",
        "_shards" : {
          "total" : 2,
          "successful" : 1,
          "failed" : 0
        },
        "_seq_no" : 0,
        "_primary_term" : 1,
        "status" : 201
      }
    },
    {
      "index" : {
        "_index" : "curl_index",
        "_id" : "2",
        "_version" : 1,
        "result" : "created",
        "_shards" : {
          "total" : 2,
          "successful" : 1,
          "failed" : 0
        },
        "_seq_no" : 1,
        "_primary_term" : 1,
        "st

In [0]:
%sh 

curl -X GET "localhost:9200/curl_index/_search?pretty"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   799    0   799    0     0   4700      0 --:--:-- --:--:-- --:--:--  4727
{
  "took" : 157,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "skipped" : 0,
    "failed" : 0
  },
  "hits" : {
    "total" : {
      "value" : 3,
      "relation" : "eq"
    },
    "max_score" : 1.0,
    "hits" : [
      {
        "_index" : "curl_index",
        "_id" : "1",
        "_score" : 1.0,
        "_source" : {
          "name" : "Bilbo",
          "age" : 50
        }
      },
      {
        "_index" : "curl_index",
        "_id" : "2",
        "_score" : 1.0,
        "_source" : {
          "name" : "Gandalf",
          "age" : 1000
        }
      },
      {
        "_index" : "curl_index",
        "_id" : "3",
        "_s

In [0]:
%sh 

curl -X GET "localhost:9200/curl_index/_doc/1?pretty"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   179  100   179    0     0   6629      0 --:--:-- --:--:-- --:--:--  6629
{
  "_index" : "curl_index",
  "_id" : "1",
  "_version" : 1,
  "_seq_no" : 0,
  "_primary_term" : 1,
  "found" : true,
  "_source" : {
    "name" : "Bilbo",
    "age" : 50
  }
}


In [0]:
%sh 

curl -X DELETE "localhost:9200/curl_index/_doc/1?pretty"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   204  100   204    0     0   5828      0 --:--:-- --:--:-- --:--:--  5828
{
  "_index" : "curl_index",
  "_id" : "1",
  "_version" : 2,
  "result" : "deleted",
  "_shards" : {
    "total" : 2,
    "successful" : 1,
    "failed" : 0
  },
  "_seq_no" : 3,
  "_primary_term" : 1
}


In [0]:
%sh

curl -X GET "localhost:9200/curl_index/_search/?pretty"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   621    0   621    0     0  32684      0 --:--:-- --:--:-- --:--:-- 32684
{
  "took" : 4,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "skipped" : 0,
    "failed" : 0
  },
  "hits" : {
    "total" : {
      "value" : 2,
      "relation" : "eq"
    },
    "max_score" : 1.0,
    "hits" : [
      {
        "_index" : "curl_index",
        "_id" : "2",
        "_score" : 1.0,
        "_source" : {
          "name" : "Gandalf",
          "age" : 1000
        }
      },
      {
        "_index" : "curl_index",
        "_id" : "3",
        "_score" : 1.0,
        "_source" : {
          "name" : "Thorin",
          "age" : 195
        }
      }
    ]
  }
}


In [0]:
%sh

curl -X GET "localhost:9200/curl_index/_search?q=Gan*"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   250    0   250    0     0  16666      0 --:--:-- --:--:-- --:--:-- 16666
{"took":4,"timed_out":false,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0},"hits":{"total":{"value":1,"relation":"eq"},"max_score":1.0,"hits":[{"_index":"curl_index","_id":"2","_score":1.0,"_source":{ "name": "Gandalf", "age": 1000 }}]}}

In [0]:
%sh

curl -X GET "localhost:9200/curl_index/_search?q=195"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   248    0   248    0     0  17714      0 --:--:-- --:--:-- --:--:-- 17714
{"took":6,"timed_out":false,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0},"hits":{"total":{"value":1,"relation":"eq"},"max_score":1.0,"hits":[{"_index":"curl_index","_id":"3","_score":1.0,"_source":{ "name": "Thorin", "age": 195 }}]}}

## Test elastic search in databricks

In [0]:
%sh 
nc -vz localhost 9200

Connection to localhost 9200 port [tcp/*] succeeded!


In [0]:
people = spark.createDataFrame( [ ("Bilbo",     50), 
                                  ("Gandalf", 1000), 
                                  ("Thorin",   195),  
                                  ("Balin",    178), 
                                  ("Kili",      77),
                                  ("Dwalin",   169), 
                                  ("Oin",      167), 
                                  ("Gloin",    158), 
                                  ("Fili",      82), 
                                  ("Bombur",  None)
                                ], 
                                ["name", "age"] 
                              )

In [0]:
# Overwrite the data each time

# NOTE: We **must** set the es.nodes.wan.only property to 'true' so that the connector will connect to the node(s) specified by the `es.nodes` parameter.
#       Without this setting, the ES connector will try to discover ES nodes on the network using a broadcast ping, which won't work.
#       We want to connect to the node(s) specified in `es.nodes`.

es_read_conf = {
    "es.nodes": "localhost",
    "es.port": "9200",
    "es.index.auto.create": "true",
    "es.nodes.wan.only": "true",  # Enable for WAN/Cloud instances
}

( people.write
  .format( "org.elasticsearch.spark.sql" )
  .options(**es_read_conf)
  .mode("overwrite")  # Optional: Overwrite existing data if needed
  .save("databricks_index")  # Ensure index name is correct
)

In [0]:
# NOTE: We **must** set the es.nodes.wan.only property to 'true' so that the connector will connect to the node(s) specified by the `es.nodes` parameter.
#       Without this setting, the ES connector will try to discover ES nodes on the network using a broadcast ping, which won't work.
#       We want to connect to the node(s) specified in `es.nodes`.
df = (spark.read
      .format( "org.elasticsearch.spark.sql" )
      .option( "es.nodes",   "localhost" )
      .option( "es.port",    9200     )
      #.option( "es.net.ssl", ssl      )
      .option( "es.nodes.wan.only", "true" )
      #.option("es.query", "?q=*o*") # filter on all columns for contains "o"
      #.option("es.query", '{"query": {"term": {"age": 77}}}')  # filter on equals 77
      #.option("es.query", '{"query": {"range": {"age": {"gt": 77}}}}')  # Filter where 'age' is greater than 77
      .load( f"databricks_index" )#.save( f"{index}" )
     )

display(df)

age,name
50,Bilbo
169,Dwalin
178,Balin
77,Kili
167,Oin
82,Fili
null,Bombur
195,Thorin
1000,Gandalf
158,Gloin


In [0]:
# NOTE: We **must** set the es.nodes.wan.only property to 'true' so that the connector will connect to the node(s) specified by the `es.nodes` parameter.
#       Without this setting, the ES connector will try to discover ES nodes on the network using a broadcast ping, which won't work.
#       We want to connect to the node(s) specified in `es.nodes`.
df = (spark.read
      .format( "org.elasticsearch.spark.sql" )
      .option( "es.nodes",   "localhost" )
      .option( "es.port",    9200     )
      #.option( "es.net.ssl", ssl      )
      .option( "es.nodes.wan.only", "true" )
      .option("es.query", "?q=*o*") # filter on all columns for contains "o"
      #.option("es.query", '{"query": {"term": {"age": 77}}}')  # filter on equals 77
      #.option("es.query", '{"query": {"range": {"age": {"gt": 77}}}}')  # Filter where 'age' is greater than 77
      .load( f"databricks_index" )#.save( f"{index}" )
     )

display(df)

age,name
50,Bilbo
167,Oin
null,Bombur
195,Thorin
158,Gloin


In [0]:
# NOTE: We **must** set the es.nodes.wan.only property to 'true' so that the connector will connect to the node(s) specified by the `es.nodes` parameter.
#       Without this setting, the ES connector will try to discover ES nodes on the network using a broadcast ping, which won't work.
#       We want to connect to the node(s) specified in `es.nodes`.
df = (spark.read
      .format( "org.elasticsearch.spark.sql" )
      .option( "es.nodes",   "localhost" )
      .option( "es.port",    9200     )
      #.option( "es.net.ssl", ssl      )
      .option( "es.nodes.wan.only", "true" )
      #.option("es.query", "?q=*o*") # filter on all columns for contains "o"
      .option("es.query", '{"query": {"term": {"age": 77}}}')  # filter on age equals 77
      #.option("es.query", '{"query": {"range": {"age": {"gt": 77}}}}')  # Filter where 'age' is greater than 77
      .load( f"databricks_index" )#.save( f"{index}" )
     )

display(df)

age,name
77,Kili


In [0]:
# NOTE: We **must** set the es.nodes.wan.only property to 'true' so that the connector will connect to the node(s) specified by the `es.nodes` parameter.
#       Without this setting, the ES connector will try to discover ES nodes on the network using a broadcast ping, which won't work.
#       We want to connect to the node(s) specified in `es.nodes`.
df = (spark.read
      .format( "org.elasticsearch.spark.sql" )
      .option( "es.nodes",   "localhost" )
      .option( "es.port",    9200     )
      #.option( "es.net.ssl", ssl      )
      .option( "es.nodes.wan.only", "true" )
      #.option("es.query", "?q=*o*") # filter on all columns for contains "o"
      #.option("es.query", '{"query": {"term": {"age": 77}}}')  # filter on equals 77
      .option("es.query", '{"query": {"range": {"age": {"gt": 77}}}}')  # Filter where 'age' is greater than 77
      .load( f"databricks_index" )#.save( f"{index}" )
     )

display(df)

age,name
169,Dwalin
178,Balin
167,Oin
82,Fili
195,Thorin
1000,Gandalf
158,Gloin


In [0]:
# Creates a Delta table called table_name
#df.write.format("delta").saveAsTable(table_name)